<a href="https://colab.research.google.com/github/ven-diagram/COMP61332_Coursework/blob/main/DepPath_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DepPath-SVM

This is the notebook for an SVM focused on using dependency paths and ensemble learningfor relation extraction

In [1]:
# Clone the WebNLG dataset

!git clone https://gitlab.com/shimorina/webnlg-dataset.git

Cloning into 'webnlg-dataset'...
remote: Enumerating objects: 5112, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 5112 (delta 2), reused 0 (delta 0), pack-reused 5106 (from 1)
Receiving objects: 100% (5112/5112), 26.09 MiB | 15.40 MiB/s, done.
Resolving deltas: 100% (4010/4010), done.
Updating files: 100% (1425/1425), done.


In [2]:
# Clone the WebNLG toolkit

!git clone https://github.com/WebNLG/webnlg_toolkit.git
%cd webnlg_toolkit
!pip install -e .

Cloning into 'webnlg_toolkit'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 230 (delta 45), reused 211 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (230/230), 16.78 MiB | 15.77 MiB/s, done.
Resolving deltas: 100% (45/45), done.
/content/webnlg_toolkit
Obtaining file:///content/webnlg_toolkit
  Preparing metadata (setup.py) ... done
  Running setup.py develop for webnlg_toolkit


In [3]:
# Useful imports

import os
import glob
import json
import numpy as np
import pandas as pd
import torch
import re
import spacy

from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split

!pip install datasets
from datasets import Dataset as HFDataset
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import webnlg_toolkit
from webnlg_toolkit.utils.data import load_webnlg_dataset
from webnlg_toolkit.utils.data import load_webnlg_xml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.8 MB/s eta 0:00:00


# Pre-processing the data

The next section will focus on pre-processing the data and getting it in the form required by extracting the sentences, relations and entities into a training and test dataframe.

In [51]:
# Define paths
dataset_root = "/content/webnlg-dataset/release_v3.0/en/train/"
dev_root = "/content/webnlg-dataset/release_v3.0/en/dev/"
test_data = load_webnlg_dataset("/content/webnlg-dataset/release_v3.0/en/test/rdf-to-text-generation-test-data-with-refs-en.xml", task="rdf2text")

# Function to find dataset files
def find_dataset_files(root_folder, file_extension=".xml"):
    dataset_files = []
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith(file_extension):
                dataset_files.append(os.path.join(subdir, file))
    return dataset_files

# Load training data
def load_all_data(root_folder):
    dataset_files = find_dataset_files(root_folder, file_extension=".xml")
    all_data = []
    for file in dataset_files:
        print(f"Loading: {file}")
        data = load_webnlg_dataset(file, task="rdf2text")
        # print(data)
        all_data.extend(data)
    return all_data

# Load training and test data
train_data = load_all_data(dataset_root)
dev_data = load_all_data(dev_root)

# Convert to DataFrames
train_df = pd.DataFrame(train_data, columns=["input", "output"])
dev_df = pd.DataFrame(dev_data, columns=["input", "output"])
test_df = pd.DataFrame(test_data, columns=["input", "output"])
print(f"Training examples: {len(train_df)}")
print(f"Dev examples: {len(dev_df)}")
print(f"Test examples: {len(test_df)}")

# Display some examples
print("\nTraining examples:")
print(train_df.head(3))

print("\nDev examples:")
print(dev_df.head(3))

print("\nTest examples:")
print(test_df.head(3))

Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/CelestialBody.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/MeanOfTransportation.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/University.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/Politician.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/Athlete.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/ComicsCharacter.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/Food.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/SportsTeam.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/Company.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/Astronaut.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/WrittenWork.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/4triples/Airport.xml
Loading: /content/webnlg-dataset/release_v3.0/en/

In [41]:
# Extract the relations from the file in the format <P> relation <O>
def extract_relations(input_text):
    relations = []
    matches = re.findall(r'<P>\s(.*?)\s<O>', input_text)
    return matches

# Apply relation extraction to create a new column with relation types
train_df['relations'] = train_df['input'].apply(extract_relations)
# dev_df['relations'] = dev_df['input'].apply(extract_relations)
test_df['relations'] = test_df['input'].apply(extract_relations)

print(train_df['relations'])

0        [discoverer, epoch, orbital period, periapsis]
1        [discoverer, epoch, orbital period, periapsis]
2        [discoverer, epoch, orbital period, periapsis]
3              [discoverer, epoch, periapsis, apoapsis]
4              [discoverer, epoch, periapsis, apoapsis]
                              ...                      
35421                                [leader, location]
35422                                [leader, location]
35423                                [leader, location]
35424                          [leader title, location]
35425                          [leader title, location]
Name: relations, Length: 35426, dtype: object


In [53]:
# Extracts the entities from an input in the format <S> entity1 <P> relation <O> entity2
def extract_entities(input_text):
    entity_pairs = []
    matches = re.findall(r'<S>\s(.*?)\s<P>.*?<O>\s(.*?)$', input_text)
    return matches

# Apply entity extraction to create a new column with entity pairs
train_df['entity_pairs'] = train_df['input'].apply(extract_entities)
dev_df['entity_pairs'] = dev_df['input'].apply(extract_entities)
test_df['entity_pairs'] = test_df['input'].apply(extract_entities)

print(train_df['entity_pairs'])

0                  [((15788) 1993 SB, 3997100000000.0)]
1                  [((15788) 1993 SB, 3997100000000.0)]
2                  [((15788) 1993 SB, 3997100000000.0)]
3        [((15788) 1993 SB, 7715100000.0 (kilometres))]
4        [((15788) 1993 SB, 7715100000.0 (kilometres))]
                              ...                      
35421            [(250 Delaware Avenue, United States)]
35422            [(250 Delaware Avenue, United States)]
35423            [(250 Delaware Avenue, United States)]
35424            [(250 Delaware Avenue, United States)]
35425            [(250 Delaware Avenue, United States)]
Name: entity_pairs, Length: 35426, dtype: object


In [43]:
# Check we have loaded the correct amount of relations
all_relations = []
for relations in train_df['relations']:
    all_relations.extend(relations)
unique_relations = sorted(list(set(all_relations)))

print(f"\nNumber of unique relation types: {len(unique_relations)}")
print("Example relation types:", unique_relations[:5])


Number of unique relation types: 372
Example relation types: [' l c c n_number', ' national register of historic places reference number', '1st runway length feet', '1st runway length metre', '1st runway number']


# Extracting dependency paths

We extract the dependency path of the entities using spaCys English NER, parser, tagger and tokeniser.

In [44]:
# Load a natural language processor using spaCy
nlp = spacy.load("en_core_web_sm")

# Extract the dependency path of entity1 and entity2
def extract_dependency_path(input_text, entity1, entity2):
    doc = nlp(input_text)
    dep_path = []

    for token in doc:
        if token.text in [entity1, entity2]:
            dep_path.append(token.dep_)

    return " ".join(dep_path)

train_df["dep_path"] = train_df.apply(lambda row: extract_dependency_path(row["output"], row["entity_pairs"][0][0], row["entity_pairs"][0][1]), axis=1)
# dev_df["dep_path"] = dev_df.apply(lambda row: extract_dependency_path(row["output"], row["entity_pairs"][0][0], row["entity_pairs"][0][1]), axis=1)
test_df['dep_path'] = test_df.apply(lambda row: extract_dependency_path(row["input"], row["entity_pairs"][0][0], row["entity_pairs"][0][1]), axis=1)

# Prepare the training and test datasets

We now add context to the input sentences such as the position of their entities and the dependency paths calculated. The relations are their corresponding labels.

In [45]:
# Create a simplified dataset for SVM
def prepare_data(df):
    X = []
    y = []

    # Add the sentences their respective relations to the datasets X and y
    for _, row in df.iterrows():
        if len(row['entity_pairs']) > 0 and len(row['relations']) > 0:
            entity1, entity2 = row['entity_pairs'][0]
            relation = row['relations'][0]
            sentence = row['input']

            # Highlight entities to preserve context
            sentence = sentence.replace(entity1, f"<E1> {entity1} </E1>")
            sentence = sentence.replace(entity2, f"<E2> {entity2} </E2>")

            # Concatenate on the dependency path to the end of the sentence to add further context to the sentence
            sentence += row["dep_path"]

            X.append(sentence)
            y.append(relation)

    return X, y

# Prepare data
X_train, y_train = prepare_data(train_df)
X_test, y_test = prepare_data(test_df)

print(f"SVM training examples: {len(X_train)}")
print(f"SVM test examples: {len(X_test)}")

SVM training examples: 35426
SVM test examples: 5150


# The pipeline

We use a voting classifer to find the best output class given the output of three SVMs with different kernels. We use a TfidVectorizer to vectorize the main sentence that is fed, and use the countVectorizer to target the dependency paths.

In [10]:
# Define three seperate support vector classifiers with different kernels
spatial_svm = SVC(kernel='linear')
temporal_svm = SVC(kernel='rbf')
causal_svm = SVC(kernel='poly')

# Define a voting Classifier with hard voting that will output the class with the highest majority of votes
ensemble_svm = VotingClassifier(estimators=[
    ('spatial', spatial_svm),
    ('temporal', temporal_svm),
    ('causal', causal_svm)
], voting='hard')

# Create SVM pipeline
svm_pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tfidf_text', TfidfVectorizer(max_features=5000)),  # Main sentence text
        ('count_dep', CountVectorizer()),  # Dependency paths
    ])),
    ('svm_ensemble', ensemble_svm)
])

In [11]:
# Train SVM
print("Training SVM model...")
svm_pipeline.fit(X_train, y_train)

Training SVM model...


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tfidf_text',
                                                 TfidfVectorizer(max_features=5000)),
                                                ('count_dep',
                                                 CountVectorizer())])),
                ('svm_ensemble',
                 VotingClassifier(estimators=[('spatial', SVC(kernel='linear')),
                                              ('temporal', SVC()),
                                              ('causal',
                                               SVC(kernel='poly'))]))])

In [12]:
%cd ..
import pickle

# Save the model
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(svm_pipeline, f)

/content


# Evaluate the SVM

We use the f1-score to evaluate the SVM. other statistics are provided to verify the model is running as expected.

In [46]:
# Evaluate SVM
svm_preds = svm_pipeline.predict(X_test)
svm_accuracy = (svm_preds == y_test).mean()
svm_f1 = f1_score(y_test, svm_preds, average='weighted')

print(f"SVM Accuracy: {svm_accuracy:.4f}")
print(f"SVM F1 Score: {svm_f1:.4f}")

SVM Accuracy: 0.3041
SVM F1 Score: 0.2778


In [47]:
# Classification report for SVM
print("\nSVM Classification Report:")
print(classification_report(y_test, svm_preds))


SVM Classification Report:
                                            precision    recall  f1-score   support

                   1st runway surface type       0.00      0.00      0.00         0
                        absolute magnitude       0.33      0.50      0.40         6
                   active years start year       0.50      1.00      0.67        12
                                   address       0.00      0.00      0.00        15
                               affiliation       0.10      0.20      0.13        15
                                     album       0.00      0.00      0.00         5
                                alma mater       0.20      1.00      0.34        32
                          alternative name       0.00      0.00      0.00         0
                                  apoapsis       1.00      0.07      0.13        71
                                 architect       1.00      0.07      0.12        91
                                 area code     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [48]:
from sklearn.metrics import precision_score, recall_score

# Compute macro and weighted precision & recall with zero_division handling
macro_precision = precision_score(y_test, svm_preds, average="macro", zero_division=1)
macro_recall = recall_score(y_test, svm_preds, average="macro", zero_division=1)

weighted_precision = precision_score(y_test, svm_preds, average="weighted", zero_division=1)
weighted_recall = recall_score(y_test, svm_preds, average="weighted", zero_division=1)

print(macro_precision, macro_recall, weighted_precision, weighted_recall)

0.6834660975174104 0.3633845565822887 0.7051524427732095 0.30407766990291263


# Real-time inference for SVM relation extraction

A user may input two entities and the output will be the the given relation that the svm has extracted. The user may pick a random entity pairs from the WebNLG test set, or choose their own.

In [16]:
# Function for SVM real-time inference
def svm_extract_relation(entity1, entity2):
    # Create input text
    text = f"{entity1} {entity2}"

    # Make prediction
    prediction = svm_pipeline.predict([text])

    return prediction

In [61]:
# Use developer dataset as example set

# Collect all unique enitity pairs
dev_df_entity_pairs = set(tuple(x) for x in dev_df['entity_pairs'])
dev_df_entity_pairs = list(dev_df_entity_pairs)

# Function that lets user choose a pair of entities from example dataset
def example_entity_pair(num):
    if num < 1 or num > len(dev_df_entity_pairs):
        raise ValueError(f"Invalid index: {num}. Must be between 1 and {len(dev_df_entity_pairs)}.")

    entity1, entity2 = dev_df_entity_pairs[num - 1][0]
    return entity1, entity2

# Total number of possible example entity pairs
print(len(dev_df_entity_pairs))

1127


In [57]:
import pickle

with open('svm_model.pkl', 'rb') as f:
    svm_pipeline = pickle.load(f)

# User may input a number here to get entity pairs
entity1, entity2 = example_entity_pair(8)

# Or uncomment here to manually input their own enitity pairs
# entity1 = "Georgia (US State)"
# entity2 = "United States"

print(f"Entity 1: {entity1}\nEntity 2: {entity2}")
svm_relation = svm_extract_relation(entity1, entity2)
print(f"Extracted relation: {svm_relation}")

Entity 1: School of Business and Social Sciences at the Aarhus University
Entity 2: European University Association
Extracted relation: ['affiliation']


In [62]:
# To demonstrate the frequency of relations the SVM predicts

# from collections import Counter

# total_rels = []
# for i in range (1, 1128):
#   entity1, entity2 = example_entity_pair(i)
#   svm_relation = svm_extract_relation(entity1, entity2)
#   total_rels.append(svm_relation)

# # Print result
# flattened_rels = [arr[0] for arr in total_rels]

# # Count occurrences
# rel_count = dict(Counter(flattened_rels))

# # Print result
# sort_rel_count = dict(sorted(rel_count.items(), key=lambda item: item[1], reverse=True))

# # Print result
# print(sort_rel_count)